In [ ]:
import sys
sys.path.append("/Users/johannes/Physik/Herwig/opt/lib/python2.7/site-packages")
import yoda
import rivet
from matplotlib import pyplot as plt
import matplotlib
from matplotlib import rc,gridspec
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
matplotlib.rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
matplotlib.rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'
from matplotlib import pyplot as plt
import numpy as np


font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 20}

matplotlib.rc('font', **font)
pltpars=rivet.mkStdPlotParser()
variations=[]

#fig_partnerfinder/11/LHC-MB.yoda:Title="Max,  Different" \
#fig_partnerfinder/01/LHC-MB.yoda:Title="Rand, Different" \
#fig_partnerfinder/10/LHC-MB.yoda:Title="Max,  Partner" \
#fig_partnerfinder/00/LHC-MB.yoda:Title="Rand, Partner" 

y=yoda.readYODA("/Users/johannes/Physik/my-paper/improvingmpis/runs/fig_partnerfinder/00/LHC-MB.yoda")
variations+=[yoda.readYODA("/Users/johannes/Physik/my-paper/improvingmpis/runs/fig_partnerfinder/00/LHC-MB.yoda")]
variations+=[yoda.readYODA("/Users/johannes/Physik/my-paper/improvingmpis/runs/fig_partnerfinder/10/LHC-MB.yoda")]
variations+=[yoda.readYODA("/Users/johannes/Physik/my-paper/improvingmpis/runs/fig_partnerfinder/01/LHC-MB.yoda")]
variations+=[yoda.readYODA("/Users/johannes/Physik/my-paper/improvingmpis/runs/fig_partnerfinder/11/LHC-MB.yoda")]

variationlabels=["Rand, Partner",
                 "Max,  Partner",
                 "Rand, Different",
                 "Max,  Different"]
variationcolor=["red","blue","teal","orange"]



In [ ]:

fig=plt.figure(figsize=[13,13])
# Possibility to make ratio plots (for now only one main plot):
gs = gridspec.GridSpec(1, 1,width_ratios= [1],height_ratios= [1])
gs.update(wspace=0.0, hspace=0.0)
axmain     = plt.subplot(gs[0, 0])

n=0
for x in y:
    
    usethis=False
    if "RAW" not in x:
        header=pltpars.getHeaders(x)
        if 'Title' in header:
            if "Charged particle $\eta$" in  header['Title']:
                usethis=True
    if not usethis:
        continue
    n+=1
    #print header['Title']
    plt.ylabel(r"%s"%header['YLabel'].replace("\\text","\\mathrm"))
    plt.xlabel(r"%s"%header['XLabel'].replace("\\text","\\mathrm"))
    plt.title(r"Charged particle $\eta$ at 7\,TeV")
    data=yoda.readYODA('/Users/johannes/Physik/Herwig/opt/share/Rivet/%s.yoda'%
                       x.split("/")[1]) 
    hist=y[x]
    varhists=[]
    for var in variations:
        varhists+=[var[x]]

    datahist=data["/REF"+x]
    if n==1: 
        k=0
        for var in varhists:
            plt.step(var.xVals(),var.yVals(),marker="",c=variationcolor[k],where="mid",label=variationlabels[k])
            k+=1
    else:
        k=0
        for var in varhists:
            plt.step(var.xVals(),var.yVals(),marker="",c=variationcolor[k],where="mid",label="")
            k+=1
                
    plt.errorbar(datahist.xVals(),datahist.yVals(),
                 yerr=(datahist.yMaxs()-datahist.yMins())/2,
                 xerr=(datahist.xMaxs()-datahist.xMins())/2,
                 ls="",c="black")
    plt.text(datahist.xVals()[25], 
             max(datahist.yVals())+.5, 
             r"%s"%header['Title'].replace("Charged particle $\\eta$ at 7\\,TeV,","").replace("\\text","\\mathrm"),
             verticalalignment='top',  horizontalalignment='center',
             transform=axmain.transData , color='green')

plt.grid(linestyle='dotted')
axmain.yaxis.set_tick_params(direction="in",which="both",)
axmain.xaxis.set_tick_params(direction="in",which="both")


plt.legend(frameon=False,loc="upper right",ncol=2)

plt.ylim([0,12])
plt.show()
    
fig.savefig("partnerfinder.pdf",bbox_inches='tight')